In [102]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [103]:
actual = pd.read_pickle('actual.pkl')
# predictions = pd.read_pickle('predictions.pkl')

file_paths = actual['file_path']
arousal = actual['arousal']
valence = actual['valence']

In [108]:
data = pd.read_table('ready_models_video/results/AEmotion_Ivan_result_valid.txt', delim_whitespace=True, names = ['file_path', 'valence', 'arousal'])

In [109]:
# model = 'Audio_M_1'
# f = open('ready_models_video/results/AEmotion_Ivan_result_test.txt', 'rb')
# data = pickle.load(f)
# f.close()

In [110]:
# pred = pd.DataFrame(data, columns=['valence_{}'.format(model), 'arousal_{}'.format(model)])
# pred = pd.concat([file_paths, pred], axis = 1)

In [111]:
# predictions.merge(pred, on = 'file_path') #add inplace

In [83]:
# def get_clip(path):
#     file = path.split('\\')[:-1]
#     return '\\'.join(file)

In [113]:
# file_paths = file_paths.apply(get_clip).unique()

# valence_df = pd.DataFrame(data[:,0], columns = ['valence'])

# files = pd.DataFrame(file_paths, columns = ['file_path'])

# arousal_df = pd.DataFrame(data[:,1], columns = ['arousal'])

In [85]:
# pred_final = pd.concat([files, valence_df, arousal_df], axis = 1)

In [115]:
data.to_pickle('audio_Ivan.pkl')

In [86]:
actual['file_path'] = actual['file_path'].apply(get_clip)

clip_actual_valence = actual.groupby('file_path', sort = False)['valence'].mean()
valence_df = pd.DataFrame(clip_actual_valence.values, columns = ['valence'])

files = pd.DataFrame(list(actual.groupby('file_path', sort = False).groups.keys()), columns = ['file_path'])

clip_actual_arousal = actual.groupby('file_path', sort = False)['arousal'].mean()
arousal_df = pd.DataFrame(clip_actual_arousal.values, columns = ['arousal'])

In [87]:
actual_final = pd.concat([files, valence_df, arousal_df], axis = 1)

In [528]:
# actual_final.to_pickle('actual_mean.pkl')
# pred_final.to_pickle('pred_mean.pkl')

In [63]:
def error(actual, predict, class_num):
    pred = predict[predict.columns[class_num]].values
    act = actual[actual.columns[class_num]].values
    all = len(act)
    pos = 0
    for i in range(len(act)):
        a = act[i]
        p = pred[i]
        if abs(a - p) < 0.1:
            pos += 1
    return pos/all

In [89]:
fin = actual_final.merge(pred_final, on = 'file_path')
actual = fin[['valence_x', 'arousal_x']]
pred = fin[['valence_x', 'arousal_x']]

In [98]:
# print('valence error by frames = {}'.format(error(actual, pred, 1)))
# print('arousal error by frames = {}'.format(error(actual, pred, 2)))

In [99]:
print('valence error by frames = {}'.format(error(actual_final, pred_final, 1)))
print('arousal error by frames = {}'.format(error(actual_final, pred_final, 2)))

valence error by frames = 0.24778761061946902
arousal error by frames = 0.20353982300884957


In [101]:
pred_final.to_pickle('mean_pred_audio.pkl')

In [497]:
# from scipy.stats import pearsonr
# import torch
# import torch.nn.functional as F

# pred_V = pred[pred.columns[1]].values.reshape(len(pred.values[:,1]),1)
# pred_A = pred[pred.columns[2]].values.reshape(len(pred.values[:,2]),1)
# predict = np.hstack([pred_V, pred_A])

# actual_V = actual[actual.columns[1]].values.reshape(len(actual.values[:,1]),1)
# actual_A = actual[actual.columns[2]].values.reshape(len(actual.values[:,2]),1)
# actual = np.hstack([actual_V, actual_A])

# result = []
# for k, name in enumerate(['Valence', 'Arousal']):
#     target = torch.from_numpy(actual[:,k]).double()
#     pred = torch.from_numpy(predict[:,k]).double()
#     test_acc = torch.nonzero(F.relu(-(target - pred).abs_() + 0.1)).size(0)
#     test_acc *= 100 / actual.shape[0]

#     result.append(test_acc)
#     print('   accuracy per frames: %0.3f%%' % test_acc)